In [1]:
import pandas as pd
df = pd.read_csv("train.csv")

In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df = df.dropna()

In [8]:
X = df.drop('label', axis = 1)

In [9]:
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.11.0'

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [19]:
voc_size = 5000

In [20]:
messages=X.copy()

In [21]:
messages.reset_index(inplace=True)

In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [30]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1135, 825, 2196, 4623, 3415, 2891, 1105, 696, 4200, 1321],
 [3496, 3565, 269, 4130, 2297, 516, 4356],
 [673, 3239, 2869, 3470],
 [1212, 2006, 2756, 391, 3580, 3368],
 [3836, 2297, 2323, 2882, 899, 1540, 2297, 4804, 2425, 2260],
 [4907,
  4898,
  1164,
  236,
  3567,
  2737,
  4481,
  2293,
  351,
  4376,
  4165,
  4505,
  4405,
  3528,
  4356],
 [4225, 2117, 2010, 2380, 1174, 2883, 76, 660, 4577, 3583, 1472],
 [4526, 2449, 4469, 1178, 4522, 3381, 2737, 1801, 4577, 3583, 1472],
 [2036, 1927, 1162, 2951, 3073, 4877, 317, 415, 2737, 3801],
 [2515, 4202, 4315, 3634, 3653, 3459, 309, 1890],
 [128, 316, 2354, 2241, 4789, 3100, 1415, 751, 2082, 3144, 3575],
 [391, 2957, 3415, 4877, 2737, 4522],
 [4649, 2130, 2921, 2501, 3228, 4603, 2407, 2566, 372],
 [4258, 889, 3882, 1824, 623, 4833, 756, 4577, 3583, 1472],
 [4813, 999, 4474, 970, 2249, 4577, 3583, 1472],
 [4223, 1470, 4369, 1449, 4644, 4083, 1457, 828, 382, 2394],
 [4437, 462, 3565],
 [1981, 615, 1219, 3627, 2737, 4238, 1513, 4356],
 [369

In [32]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  696 4200 1321]
 [   0    0    0 ... 2297  516 4356]
 [   0    0    0 ... 3239 2869 3470]
 ...
 [   0    0    0 ... 4577 3583 1472]
 [   0    0    0 ...  121 1174 2798]
 [   0    0    0 ... 1294 1210 2512]]


In [33]:
len(embedded_docs)

18285

In [35]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [39]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 16ms/step - loss: 0.3264 - accuracy: 0.8570 - val_loss: 0.2003 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 3s 16ms/step - loss: 0.1375 - accuracy: 0.9475 - val_loss: 0.1934 - val_accuracy: 0.9223
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0947 - accuracy: 0.9643 - val_loss: 0.2259 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0732 - accuracy: 0.9742 - val_loss: 0.2432 - val_accuracy: 0.9171
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0527 - accuracy: 0.9831 - val_loss: 0.3133 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0399 - accuracy: 0.9881 - val_loss: 0.2993 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0284 - accuracy: 0.9908 - val_loss: 0.3707 - val_accuracy: 0.9029

In [41]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

189/189 [==============================] - 1s 3ms/step


array([[3137,  282],
       [ 268, 2348]], dtype=int64)